# **Реализация игры**


In [19]:
from itertools import product
import random

RANKS = {'2', '3', '4', '5', '6', '7', '8', '9', '10', 'jack', 'queen', 'king', 'ace'}
SUITS = {"\u2665", "\u2660", "\u2666", "\u2663"}


class GameForGA:

    def __init__(self):
        self.deck = self.create_deck()
        random.shuffle(self.deck)

        self.player_hand = []
        self.dealer_hand = []

    @staticmethod
    def create_deck():
        deck = []
        for suit, rank in product(SUITS, RANKS):
            deck.append([rank, suit])
        return deck

    def get_card(self):
        return self.deck.pop()

    def start_game(self):
        self.player_hand = [self.get_card(), self.get_card()]
        self.dealer_hand = [self.get_card(), self.get_card()]

    @staticmethod
    def hand_value(hand):
        value = 0
        aces = 0
        for card in hand:
            if card[0] == 'ace':
                value += 11
                aces += 1
            elif card[0].isdigit():
                value += int(card[0])
            else:
                value += 10
        while value > 21 and aces > 0:
            value -= 10
            aces -= 1
        return value

    @staticmethod
    def print_hand(hand):
        res = ""
        for card in hand:
            res += card[0] + ' ' + card[1] + ' '
        return res

    def player_action(self, action):
        if action == "h":
            self.player_hand.append(self.get_card())
        return self.game_status()

    def dealer_action(self):
        while self.hand_value(self.dealer_hand) < 17:
            self.dealer_hand.append(self.get_card())
            # print("Dealer hits card and has:", self.print_hand(self.dealer_hand), self.hand_value(self.dealer_hand))

    def game_status(self):
        player_value = self.hand_value(self.player_hand)
        if player_value > 21:
            # return "player BUST"
            return 'l'
        elif player_value == 21:
            # return "player BLACKJAСK"
            return 'w'
        else:
            return "continue"

    def game_result(self):
        player_value = self.hand_value(self.player_hand)
        dealer_value = self.hand_value(self.dealer_hand)

        if player_value > 21:
            # return f"player have {player_value} \nLOSS"
            return 'l'
        elif dealer_value > 21 or player_value > dealer_value:
            # return f"player have {player_value} dealer have {dealer_value} \nWIN"
            return 'w'
        elif player_value == dealer_value:
            # return "equal"
            return 'e'
        else:
            # return f"player have {player_value} dealer have {dealer_value} \nLOSS"
            return 'l'

In [20]:
def playGA(strategy):
    g = GameForGA()
    g.start_game()
    type_of_hand = 0
    help_type = 1
    # print("Dealer shows:", g.print_hand(g.dealer_hand[:1]))
    g.print_hand(g.dealer_hand[:1])
    status = "continue"
    while status == "continue":
        # print(g.print_hand(g.player_hand), 'your value:', g.hand_value(g.player_hand))
        g.print_hand(g.player_hand)
        g.hand_value(g.player_hand)

        if help_type == 1:
            if (g.player_hand[0][0] == 'ace' or g.player_hand[1][0] == 'ace'):
                type_of_hand = 1
            if (g.player_hand[0][0] == g.player_hand[1][0]):
                type_of_hand = 2
        help_type = 0
        if type_of_hand != 0:
            break
        if g.hand_value(g.player_hand) == 21 or g.hand_value(g.dealer_hand) == 21:
            # if g.hand_value(g.player_hand) != g.hand_value(g.dealer_hand):
            # print('BLACKJAСK')
            status = 'BLACKJAСK'

        if status == 'BLACKJAСK':
            break
        # action = input("Enter an action (h/s): ") это для игры в ручную
        # action = strategy[type_of_hand][g.hand_value(g.dealer_hand[:1])][g.hand_value(g.player_hand)] для 3 раздач

        action = strategy[g.hand_value(g.player_hand) - 5][
            g.hand_value(g.dealer_hand[:1]) - 2]  # только для жесткой раздачи
        status = g.player_action(action)

        if action == "s":
            break

    if status == "continue":
        # print("Dealer has:", g.print_hand(g.dealer_hand), g.hand_value(g.dealer_hand))
        g.print_hand(g.dealer_hand)
        g.hand_value(g.dealer_hand)
        g.dealer_action()
    if type_of_hand == 0:
        return g.game_result()
    else:
        return 'e'  # сделали так, чтобы рассматривать только жесткие раздачи (не меняем остаток банка)
#основная часть закомментированного кода - реализация пользовательского интерфейса игры 

# **Реализация генетического алгоритма**


In [21]:
import pandas as pd
import random
#from game import playGA
import matplotlib.pyplot as plt
import copy
import multiprocessing as mp

from loguru import logger


def creator_df():
    df_hard = pd.DataFrame(index=range(5, 21), columns=range(2, 12))
    # df_soft = pd.DataFrame(index=range(13, 21), columns=range(2, 12))
    # df_double = pd.DataFrame(index=range(2, 21, 2), columns=range(2, 12))
    for i in range(5, 21):
        for j in range(2, 12):
            df_hard.loc[i, j] = random.choice(['h', 's'])
    # for i in range(13, 21):
    #     for j in range(2, 12):
    #         df_soft.loc[i, j] = random.choice(['h', 's'])
    # for i in range(2, 21, 2):
    #     for j in range(2, 12):
    #         df_double.loc[i, j] = random.choice(['h', 's'])
    df_hard = df_hard.values.tolist()
    return df_hard  # , df_soft, df_double
#закомментированный код в creator_df - создание случайной стратегии для мягких и парных рук


class FitnessMax():
    def __init__(self):
        self.values = 0


class Individual(list):
    def __init__(self, *args):
        super().__init__(*args)
        self.fitness = FitnessMax()


def blackjack_fitness(individual):
    cash = 50000
    dep = 1
    for _ in range(50000):
        result = playGA(individual)
        if result == 'l':
            cash -= dep
        elif result == 'w':
            cash += dep
        else:
            cash += 0
    return cash / 50000


def individual_creator():
    return Individual(creator_df())


def population_creator(n):
    return [individual_creator() for i in range(n)]


def tournament(population):
    offspring = []
    for i in range(0, 450, 2):
        offspring.append(max([population[i], population[i + 1]], key=lambda ind: ind.fitness.values))
    sorted_offspring = sorted(offspring, key=lambda ind: ind.fitness.values, reverse=True)
    return sorted_offspring[:200]


def crossing(parent_1, parent_2):
    child_1 = copy.deepcopy(parent_1)
    child_2 = copy.deepcopy(parent_2)
    for i in range(len(child_1)):
        s = random.randint(len(child_1[0]) // 2 + 1, len(child_1[0]) - 1)
        p = random.randint(1, s)
        child_1[i][p:s], child_2[i][p:s] = child_2[i][p:s], child_1[i][p:s]

    return child_1, child_2


def mutation(parent):
    mutant = copy.deepcopy(parent)
    for i in range(len(mutant)):
        for j in range(len(mutant[0])):
            if random.random() < 0.05:
                mutant[i][j] = 'h' if mutant[i][j] == 's' else 's'
    return mutant

In [ ]:
logger.info("Start. Create population")
population = population_creator(450)
logger.info("Population created")
generationCounter = 0

logger.info("Start BlackJackFitness")
fitnessValues = list(map(blackjack_fitness, population))
logger.info("End BlackJackFitness")

for individual, fitnessValue in zip(population, fitnessValues):
    individual.fitness.values = fitnessValue

maxFitnessValue = []
meanFitnessValue = []

fitnessValues = [individual.fitness.values for individual in population]

while generationCounter < 300:
    generationCounter += 1
    offspring = tournament(population)
    # print(len(offspring))

    for i in range(0, len(offspring), 2):
        cross_1, cross_2 = crossing(offspring[i], offspring[i + 1])
        offspring.append(cross_1)
        offspring.append(cross_2)

    # print(len(offspring))
    for i in range(0, len(offspring), 8):
        mutantik = mutation(offspring[i])
        offspring.append(mutantik)
    # print(len(offspring))

    freshFitnessValues = list(map(blackjack_fitness, offspring))
    for individual, fitnessValue in zip(offspring, freshFitnessValues):
        individual.fitness.values = fitnessValue

    population[:] = offspring
    # print(len(population))
    fitnessValues = [ind.fitness.values for ind in population]

    maxFitness = max(fitnessValues)
    meanFitness = sum(fitnessValues) / len(population)
    maxFitnessValue.append(maxFitness)
    meanFitnessValue.append(meanFitness)
    print(f"Поколение {generationCounter}: Макс. приспособленность = {maxFitness}, Средняя = {meanFitness}")

    best_i = fitnessValues.index(max(fitnessValues))
    ans = [*population[best_i]]
    df_ans = pd.DataFrame(ans, index=range(5, 21), columns=range(2, 12))
    print('Лучший инидивид \n', df_ans, '\n')

plt.plot(maxFitnessValue, label='Максимальная')
plt.plot(meanFitnessValue, label='Средняя')
plt.xlabel('Поколение')
plt.ylabel('Максимальная и средняя приспособленность')
plt.legend()
plt.show()

In [23]:
def creatorDF():
    df_hard = pd.DataFrame(index=range(5, 21), columns=range(2, 12))
    # df_soft = pd.DataFrame(index=range(13, 21), columns=range(2, 12))
    # df_double = pd.DataFrame(index=range(2, 21, 2), columns=range(2, 12))
    for i in range(5, 21):
        for j in range(2, 12):
            df_hard.loc[i, j] = random.choice(['h', 's'])
    # for i in range(13, 21):
    #     for j in range(2, 12):
    #         df_soft.loc[i, j] = random.choice(['h', 's'])
    # for i in range(2, 21, 2):
    #     for j in range(2, 12):
    #         df_double.loc[i, j] = random.choice(['h', 's'])
    df_hard = df_hard.values.tolist()
    return df_hard #, df_soft, df_double

In [26]:
df_random = creatorDF()
df_random

[['s', 's', 's', 's', 'h', 's', 'h', 's', 'h', 'h'],
 ['s', 's', 's', 'h', 's', 'h', 's', 's', 's', 's'],
 ['s', 's', 's', 's', 'h', 's', 's', 'h', 's', 's'],
 ['s', 'h', 's', 'h', 's', 'h', 'h', 'h', 's', 's'],
 ['h', 'h', 's', 'h', 'h', 'h', 'h', 'h', 's', 's'],
 ['s', 's', 'h', 'h', 's', 'h', 's', 's', 'h', 's'],
 ['h', 'h', 'h', 'h', 's', 'h', 'h', 's', 's', 'h'],
 ['h', 's', 's', 'h', 'h', 'h', 's', 'h', 'h', 's'],
 ['h', 'h', 'h', 's', 's', 's', 'h', 'h', 's', 'h'],
 ['s', 's', 'h', 's', 'h', 's', 's', 's', 'h', 'h'],
 ['h', 's', 's', 's', 's', 's', 'h', 's', 'h', 'h'],
 ['s', 's', 'h', 'h', 's', 's', 's', 's', 'h', 's'],
 ['s', 'h', 's', 'h', 'h', 'h', 'h', 'h', 's', 'h'],
 ['s', 's', 's', 'h', 'h', 's', 'h', 's', 's', 's'],
 ['s', 'h', 'h', 's', 'h', 's', 'h', 's', 'h', 'h'],
 ['s', 's', 'h', 'h', 'h', 'h', 's', 's', 'h', 's']]

In [30]:
df_ans = [['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'s', 's' , 's', 's', 's'],
      ['s' ,'s', 's' ,'s', 's' ,'s', 's' , 's', 's', 's'],
      ['s' ,'s', 's' ,'s', 's' ,'s', 's' , 's', 's', 's'],
      ['s' ,'s', 's' ,'s', 's' ,'s', 's' , 's', 's', 's']]

In [61]:
matrix4 = [
    ["h", "s", "s", "h", "s", "h", "s", "h", "s", "s"],
    ["s", "s", "s", "h", "h", "s", "h", "s", "h", "h"],
    ["h", "s", "h", "s", "h", "h", "s", "h", "s", "s"],
    ["s", "h", "h", "s", "h", "h", "h", "h", "h", "h"],
    ["h", "s", "s", "h", "h", "s", "h", "h", "s", "h"],
    ["s", "h", "h", "h", "s", "h", "h", "h", "h", "h"],
    ["h", "h", "h", "h", "h", "s", "s", "h", "s", "h"],
    ["s", "h", "s", "s", "s", "h", "h", "s", "h", "h"],
    ["s", "h", "s", "s", "s", "s", "s", "h", "s", "h"],
    ["s", "h", "s", "h", "s", "h", "s", "h", "s", "s"],
    ["s", "s", "s", "s", "s", "h", "h", "h", "h", "h"],
    ["h", "s", "h", "s", "s", "h", "h", "h", "s", "h"],
    ["s", "s", "s", "h", "h", "s", "h", "s", "s", "s"],
    ["s", "h", "s", "s", "s", "s", "s", "s", "h", "s"],
    ["s", "s", "h", "s", "s", "h", "s", "h", "s", "s"],
    ["s", "s", "s", "h", "s", "s", "s", "s", "h", "s"]
]

In [35]:
counter = 0
for i in range(len(df)):
    for j in range(len(df[0])):
        if matrix4[i][j] == df_ans[i][j]:
            counter += 1
print(counter/(len(df) * len(df[0])))

0.7375


In [40]:
matrix33 = [
    ["s", "s", "h", "s", "s", "h", "h", "h", "h", "s"],
    ["s", "h", "h", "h", "h", "h", "h", "h", "h", "h"],
    ["h", "s", "s", "h", "h", "h", "h", "h", "h", "s"],
    ["s", "h", "s", "h", "h", "h", "h", "h", "h", "s"],
    ["h", "h", "h", "h", "h", "h", "h", "h", "h", "h"],
    ["h", "h", "h", "h", "h", "h", "h", "h", "h", "s"],
    ["h", "h", "h", "h", "h", "h", "h", "h", "h", "h"],
    ["s", "h", "s", "s", "s", "h", "h", "h", "h", "h"],
    ["s", "h", "s", "s", "s", "h", "h", "h", "h", "h"],
    ["h", "h", "s", "s", "s", "h", "h", "h", "h", "s"],
    ["s", "s", "s", "s", "s", "h", "h", "h", "h", "h"],
    ["s", "s", "s", "s", "s", "h", "h", "h", "h", "s"],
    ["s", "s", "s", "s", "s", "s", "s", "s", "s", "h"],
    ["s", "s", "s", "s", "s", "s", "s", "s", "s", "s"],
    ["s", "s", "s", "s", "s", "s", "s", "s", "s", "s"],
    ["s", "s", "s", "s", "s", "s", "s", "s", "s", "s"]
]

In [41]:
counter = 0
for i in range(len(df)):
    for j in range(len(df[0])):
        if matrix33[i][j] == df_ans[i][j]:
            counter += 1
print(counter/(len(df) * len(df[0])))

0.875


In [47]:
matrix152 = [
    ["h", "s", "h", "h", "s", "h", "h", "h", "h", "s"],
    ["s", "s", "s", "h", "h", "s", "h", "h", "h", "h"],
    ["h", "s", "h", "s", "h", "h", "h", "h", "h", "s"],
    ["s", "h", "h", "h", "h", "h", "h", "h", "h", "h"],
    ["h", "h", "h", "h", "h", "h", "h", "h", "h", "h"],
    ["h", "h", "h", "h", "h", "h", "h", "h", "h", "h"],
    ["h", "h", "h", "h", "h", "h", "h", "h", "h", "h"],
    ["s", "h", "s", "s", "s", "h", "h", "h", "h", "h"],
    ["s", "h", "s", "s", "s", "h", "h", "h", "h", "h"],
    ["s", "h", "s", "s", "s", "h", "h", "h", "h", "s"],
    ["s", "s", "s", "s", "s", "h", "h", "h", "h", "h"],
    ["h", "s", "s", "s", "s", "h", "h", "h", "h", "h"],
    ["s", "s", "s", "s", "s", "s", "s", "s", "s", "s"],
    ["s", "s", "s", "s", "s", "s", "s", "s", "s", "s"],
    ["s", "s", "s", "s", "s", "s", "s", "s", "s", "s"],
    ["s", "s", "s", "s", "s", "s", "s", "s", "s", "s"]
]

In [48]:
counter = 0
for i in range(len(df)):
    for j in range(len(df[0])):
        if matrix152[i][j] == df_ans[i][j]:
            counter += 1
print(counter/(len(df) * len(df[0])))

0.9


In [55]:
def BlackJackFitness(individual, value):
    cash = value
    dep = 1
    for _ in range(value):
        result = playGA(individual)
        if result == 'l':
            cash -= dep
        elif result == 'w':
            cash += dep
        else:
            cash += 0
    return cash/value

In [62]:
BlackJackFitness(matrix4, 50000)

0.76534

In [63]:
BlackJackFitness(matrix33, 50000)

0.89288

In [59]:
BlackJackFitness(matrix152, 50000)

0.89652

In [60]:
BlackJackFitness(df_ans, 50000)

0.90828

In [65]:
matrix202 = [['s' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'h', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'s', 's' , 's', 's', 's'],
      ['s' ,'s', 's' ,'s', 's' ,'s', 's' , 's', 's', 's'],
      ['s' ,'s', 's' ,'s', 's' ,'s', 's' , 's', 's', 's'],
      ['s' ,'s', 's' ,'s', 's' ,'s', 's' , 's', 's', 's']]


In [66]:
matrix298 = [['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'s', 's' , 's', 's', 's'],
      ['s' ,'s', 's' ,'s', 's' ,'s', 's' , 's', 's', 's'],
      ['s' ,'s', 's' ,'s', 's' ,'s', 's' , 's', 's', 's'],
      ['s' ,'s', 's' ,'s', 's' ,'s', 's' , 's', 's', 's']]


In [67]:
BlackJackFitness(matrix202, 50000)

0.89598

In [68]:
BlackJackFitness(matrix298, 50000)

0.90226

In [70]:
counter = 0
for i in range(len(df)):
    for j in range(len(df[0])):
        if matrix202[i][j] == df_ans[i][j]:
            counter += 1
print(counter/(len(df) * len(df[0])))
counter = 0
for i in range(len(df)):
    for j in range(len(df[0])):
        if matrix298[i][j] == df_ans[i][j]:
            counter += 1
print(counter/(len(df) * len(df[0])))

0.9875
1.0


In [71]:
df = pd.DataFrame(df_ans, index=range(5, 21), columns=range(2, 12))
print(df)

   2  3  4  5  6  7  8  9  10 11
5   h  h  h  h  h  h  h  h  h  h
6   h  h  h  h  h  h  h  h  h  h
7   h  h  h  h  h  h  h  h  h  h
8   h  h  h  h  h  h  h  h  h  h
9   h  h  h  h  h  h  h  h  h  h
10  h  h  h  h  h  h  h  h  h  h
11  h  h  h  h  h  h  h  h  h  h
12  h  h  s  s  s  h  h  h  h  h
13  s  s  s  s  s  h  h  h  h  h
14  s  s  s  s  s  h  h  h  h  h
15  s  s  s  s  s  h  h  h  h  h
16  s  s  s  s  s  h  h  h  h  h
17  s  s  s  s  s  s  s  s  s  s
18  s  s  s  s  s  s  s  s  s  s
19  s  s  s  s  s  s  s  s  s  s
20  s  s  s  s  s  s  s  s  s  s


In [72]:
df = pd.DataFrame(matrix4, index=range(5, 21), columns=range(2, 12))
print(df)

   2  3  4  5  6  7  8  9  10 11
5   h  s  s  h  s  h  s  h  s  s
6   s  s  s  h  h  s  h  s  h  h
7   h  s  h  s  h  h  s  h  s  s
8   s  h  h  s  h  h  h  h  h  h
9   h  s  s  h  h  s  h  h  s  h
10  s  h  h  h  s  h  h  h  h  h
11  h  h  h  h  h  s  s  h  s  h
12  s  h  s  s  s  h  h  s  h  h
13  s  h  s  s  s  s  s  h  s  h
14  s  h  s  h  s  h  s  h  s  s
15  s  s  s  s  s  h  h  h  h  h
16  h  s  h  s  s  h  h  h  s  h
17  s  s  s  h  h  s  h  s  s  s
18  s  h  s  s  s  s  s  s  h  s
19  s  s  h  s  s  h  s  h  s  s
20  s  s  s  h  s  s  s  s  h  s


In [73]:
df = pd.DataFrame(matrix33, index=range(5, 21), columns=range(2, 12))
print(df)

   2  3  4  5  6  7  8  9  10 11
5   s  s  h  s  s  h  h  h  h  s
6   s  h  h  h  h  h  h  h  h  h
7   h  s  s  h  h  h  h  h  h  s
8   s  h  s  h  h  h  h  h  h  s
9   h  h  h  h  h  h  h  h  h  h
10  h  h  h  h  h  h  h  h  h  s
11  h  h  h  h  h  h  h  h  h  h
12  s  h  s  s  s  h  h  h  h  h
13  s  h  s  s  s  h  h  h  h  h
14  h  h  s  s  s  h  h  h  h  s
15  s  s  s  s  s  h  h  h  h  h
16  s  s  s  s  s  h  h  h  h  s
17  s  s  s  s  s  s  s  s  s  h
18  s  s  s  s  s  s  s  s  s  s
19  s  s  s  s  s  s  s  s  s  s
20  s  s  s  s  s  s  s  s  s  s


In [74]:
df = pd.DataFrame(matrix152, index=range(5, 21), columns=range(2, 12))
print(df)

   2  3  4  5  6  7  8  9  10 11
5   h  s  h  h  s  h  h  h  h  s
6   s  s  s  h  h  s  h  h  h  h
7   h  s  h  s  h  h  h  h  h  s
8   s  h  h  h  h  h  h  h  h  h
9   h  h  h  h  h  h  h  h  h  h
10  h  h  h  h  h  h  h  h  h  h
11  h  h  h  h  h  h  h  h  h  h
12  s  h  s  s  s  h  h  h  h  h
13  s  h  s  s  s  h  h  h  h  h
14  s  h  s  s  s  h  h  h  h  s
15  s  s  s  s  s  h  h  h  h  h
16  h  s  s  s  s  h  h  h  h  h
17  s  s  s  s  s  s  s  s  s  s
18  s  s  s  s  s  s  s  s  s  s
19  s  s  s  s  s  s  s  s  s  s
20  s  s  s  s  s  s  s  s  s  s


In [75]:
df = pd.DataFrame(matrix202, index=range(5, 21), columns=range(2, 12))
print(df)

   2  3  4  5  6  7  8  9  10 11
5   s  h  h  h  h  h  h  h  h  h
6   h  h  h  h  h  h  h  h  h  h
7   h  h  h  h  h  h  h  h  h  h
8   h  h  h  h  h  h  h  h  h  h
9   h  h  h  h  h  h  h  h  h  h
10  h  h  h  h  h  h  h  h  h  h
11  h  h  h  h  h  h  h  h  h  h
12  s  h  s  s  s  h  h  h  h  h
13  s  s  s  s  s  h  h  h  h  h
14  s  s  s  s  s  h  h  h  h  h
15  s  s  s  s  s  h  h  h  h  h
16  s  s  s  s  s  h  h  h  h  h
17  s  s  s  s  s  s  s  s  s  s
18  s  s  s  s  s  s  s  s  s  s
19  s  s  s  s  s  s  s  s  s  s
20  s  s  s  s  s  s  s  s  s  s


In [76]:
df = pd.DataFrame(matrix298, index=range(5, 21), columns=range(2, 12))
print(df)

   2  3  4  5  6  7  8  9  10 11
5   h  h  h  h  h  h  h  h  h  h
6   h  h  h  h  h  h  h  h  h  h
7   h  h  h  h  h  h  h  h  h  h
8   h  h  h  h  h  h  h  h  h  h
9   h  h  h  h  h  h  h  h  h  h
10  h  h  h  h  h  h  h  h  h  h
11  h  h  h  h  h  h  h  h  h  h
12  h  h  s  s  s  h  h  h  h  h
13  s  s  s  s  s  h  h  h  h  h
14  s  s  s  s  s  h  h  h  h  h
15  s  s  s  s  s  h  h  h  h  h
16  s  s  s  s  s  h  h  h  h  h
17  s  s  s  s  s  s  s  s  s  s
18  s  s  s  s  s  s  s  s  s  s
19  s  s  s  s  s  s  s  s  s  s
20  s  s  s  s  s  s  s  s  s  s


# **Графики и вспомогательные функции**

In [ ]:
import pandas as pd
def creatorDF():
    df_hard = pd.DataFrame(index=range(5, 21), columns=range(2, 12))
    # df_soft = pd.DataFrame(index=range(13, 21), columns=range(2, 12))
    # df_double = pd.DataFrame(index=range(2, 21, 2), columns=range(2, 12))
    for i in range(5, 21):
        for j in range(2, 12):
            df_hard.loc[i, j] = random.choice(['h', 's'])
    # for i in range(13, 21):
    #     for j in range(2, 12):
    #         df_soft.loc[i, j] = random.choice(['h', 's'])
    # for i in range(2, 21, 2):
    #     for j in range(2, 12):
    #         df_double.loc[i, j] = random.choice(['h', 's'])
    df_hard = df_hard.values.tolist()
    return df_hard #, df_soft, df_double

In [ ]:
def blackjack_fitness(individual, value):
    cash = value
    dep = 1
    for _ in range(value):
        result = playGA(individual)
        if result == 'l':
            cash -= dep
        elif result == 'w':
            cash += dep
        else:
            cash += 0
    return cash/value

In [ ]:
df1 = creatorDF()
df = [['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 'h' ,'h', 'h' ,'h', 'h' , 'h', 'h', 'h'],
      ['h' ,'h', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'h', 'h' , 'h', 'h', 'h'],
      ['s' ,'s', 's' ,'s', 's' ,'s', 's' , 's', 's', 's'],
      ['s' ,'s', 's' ,'s', 's' ,'s', 's' , 's', 's', 's'],
      ['s' ,'s', 's' ,'s', 's' ,'s', 's' , 's', 's', 's'],
      ['s' ,'s', 's' ,'s', 's' ,'s', 's' , 's', 's', 's']]

In [ ]:
import matplotlib.pyplot as plt


arr100 = [blackjack_fitness(df, 100) for i in range(100)]
arr1000 = [blackjack_fitness(df, 1000) for _ in range(100)]
arr10000 = [blackjack_fitness(df, 10000) for _ in range(100)]
arr50000 = [blackjack_fitness(df, 50000) for _ in range(100)]
arr100000 = [blackjack_fitness(df, 100000) for _ in range(100)]

arr1_100 = [blackjack_fitness(df1, 100) for i in range(100)]
arr1_1000 = [blackjack_fitness(df1, 1000) for _ in range(100)]
arr1_10000 = [blackjack_fitness(df1, 10000) for _ in range(100)]
arr1_50000 = [blackjack_fitness(df1, 50000) for _ in range(100)]
arr1_100000 = [blackjack_fitness(df1, 100000) for _ in range(100)]

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))

ax1.violinplot([arr100, arr1000, arr10000, arr50000, arr100000], showmeans=True, showmedians=True)
ax1.set_xticks([1, 2, 3, 4, 5])
ax1.set_xticklabels(['100', '1000', '10000', '50000', '100000'])
ax1.set_ylabel('Показатель пригодности')
ax1.set_xlabel('Кол-во сыгранных раздач')
ax1.set_title('Лучшая стратегия')

ax2.violinplot([arr1_100, arr1_1000, arr1_10000, arr1_50000, arr1_100000], showmeans=True, showmedians=True)
ax2.set_xticks([1, 2, 3, 4, 5])
ax2.set_xticklabels(['100', '1000', '10000', '50000', '100000'])
#ax2.set_ylabel('Показатель пригодности')
ax2.set_xlabel('Кол-во сыгранных раздач')
ax2.set_title('Случайная стратегия')

plt.show()

In [ ]:
import numpy as np
std = [np.std(arr100), np.std(arr1000),np.std(arr10000),np.std(arr50000), np.std(arr100000)]
std1 = [np.std(arr1_100), np.std(arr1_1000),np.std(arr1_10000),np.std(arr1_50000), np.std(arr1_100000)]
val = [100, 1000, 10000, 50000, 100000]

plt.figure(figsize=(10, 6))
plt.plot(val, std, marker='o', label= 'Лучшая стратегия', color = 'r')
plt.plot(val, std1, marker='s', label= 'Случайная стратегия')
plt.grid(True)
plt.legend()

plt.xscale('log', base=10)


plt.xticks(val, [f'{v:,}' for v in val], rotation=0)
plt.xlabel('Количество сыгранных раздач', fontsize=14)
plt.ylabel('Стандартное отклонение', fontsize=14)
plt.tight_layout()
plt.show()